In [1]:
import json
import numpy as np

In [2]:
def load_segments(file_path):
    with open(file_path, "r") as file:
        segments = json.load(file)
    segments = np.array(segments["segIndices"])
    segment_ids = np.unique(segments, return_inverse=True)[1]
    return segment_ids

In [7]:
def refine_segmentation(labels, segments):
    for sg in np.unique(segments):
        vote, count = np.unique(labels[segments == sg], return_counts=True)
        max_count = np.argmax(count)
        max_vote = vote[max_count]
        labels[segments == sg] = max_vote
    return labels

In [5]:
from pathlib import Path
# test_dir = Path("/home/guangda/scannet_rsync/scans_test")
test_dir = Path("/home/guangda/scannet_rsync/scans")
test_scans = list(test_dir.glob('*'))

In [13]:
import subprocess
import os
from tqdm import tqdm

for test_scan in tqdm(test_scans):
    scene_id = test_scan.name

    new_segments_path = test_scan / f"{scene_id}_vh_clean_2.0.010000.segs.json"
    original_segments_path = test_scan / f"{scene_id}_vh_clean_2.0.010000.segs.json.original"

    new_segments = load_segments(str(new_segments_path))
    original_segments = load_segments(str(original_segments_path))

    # print(new_segments.shape)

    pseudo_label = np.random.randint(low=0, high=1024, size=(new_segments.shape[0],))

    # print(new_segments, original_segments)
    original_refined_labels = refine_segmentation(labels=pseudo_label, segments=original_segments)
    new_refined_labels = refine_segmentation(labels=pseudo_label, segments=new_segments)

    if (original_refined_labels != new_refined_labels).mean() > 0.0:
        print(test_scan)

100%|██████████| 1513/1513 [05:54<00:00,  4.27it/s]


In [14]:
original_refined_labels,new_refined_labels

(array([498, 498, 498, ..., 235,  38,  38]),
 array([498, 498, 498, ..., 235,  38,  38]))